In [1]:
import pandas as pd
import numpy as np
import os
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
ignore = ['ti','510','phi__','eh','lo','tis','ho','917','72nd','120pluspound','300plus','bbecause','ohohoho','coo','woohoowooooooo','wooooo','shalalalalala','woohoohoohoooo','woohoooo','woohoowoohoooo','60s','208','mmkay','ok__','pta','umm','ja4ll','2nd','lve','uq','8th','240','ri','433','pah','ohho','oww','cos','td','illumination','r1734','z7829','dvr','1415','99','hmph','hee','bev','rds','mmmmhmm','22nd','lu','rev','uhh','unh','mmhmm','whoo','300','uhhuh','hhhow','de','30','ohh','aaaaaahhhhh','aaaaaaahhhhh ','aa','heh','ugh','300 ','quite','mmm','sort','actually','cause','gonna','lad','shh','ha','uh','ah','ahhh','ooh','ya', 'so', 'got', 'that', 'this', '000', 'em', 'huh', 'aye', 'dum', 'la', 'ssh', 'okay', 'ok', 'gotta', 'hmm', 'aw', 'ow', 'also', 'yes', 'ah', 'said', 'well', 'would', 'yeah', 'two', 'shut','till', 'shall','john', 'ya', 'gotta', 'bit', 'hi', 'outta', 'bye', 'ii', 'aah','um', 'whoa', 'wanna','wow','uh']
for x in ignore:
    STOPWORDS.add(x)

In [ ]:
# 1940-disney = 0
# 1950-disney = 1
# 1960-disney = 2
# 1970-disney = 3
# 1980-disney = 4
# 1990-disney = 5
# 2000-disney = 6
# 2010-disney = 7

# 1980-ghibli = 8
# 1990-ghibli = 9
# 2000-ghibli = 10
# 2010-ghibli = 11

# 2000-aardman = 12
# 2010-aardman = 13

# 2000-blue = 14
# 2010-blue = 15

# 2010-illumination = 16

# 1990-dreamworks = 17
# 2000-dreamworks = 18
# 2010-dreamworks = 19

# 2010-paramount = 20

# 2000-sony = 21
# 2010-sony = 22

# 1990-animation = 23
# 2000-animation = 24
# 2010-animation = 25

# 1950-disney-liveaction = 26
# 1960-disney-liveaction = 27
# 1970-disney-liveaction = 28
# 1980-disney-liveaction = 29
# 1990-disney-liveaction = 30
# 2000-disney-liveaction = 31
# 2010-disney-liveaction = 32

# 1940-disney-liveaction+animation = 34
# 1960-disney-liveaction+animation = 35
# 1970-disney-liveaction+animation = 36
# 1980-disney-liveaction+animation = 37
# 1990-disney-liveaction+animation = 38
# 2000-disney-liveaction+animation = 39
# 2010-disney-liveaction+animation = 40

In [3]:
data = []
t = []

In [45]:
path = r'C:\Users\Attilio\Desktop\progettone\word cloud subtitles\clean_subtitles(CORRETTI)\Disney\Animation + Live Action\2010'

duemila_disney_file = []
for x, y, z in os.walk(path):
    for a in z:
      #  if ".srt" in a:
        duemila_disney_file.append(x+'\\'+a)


for srt_file in duemila_disney_file:
    with open(srt_file,encoding="utf-8", errors = 'ignore') as f:
        srt_file_as_string = f.read()
    data.append(srt_file_as_string)
    t.append(40)
print(len(data))
print(len(t))

432
432


In [46]:
data1 = data
t1 = t

In [48]:
with open('tutti_i_film_1.txt', 'w',encoding="utf-8", errors = 'ignore' ) as f:
    for item in data1:
        f.write("%s\n" % item)
        f.write("##############")

In [49]:
with open('target.txt', 'w',encoding="utf-8", errors = 'ignore' ) as g:
    for item_1 in t1:
        g.write("%s\n" % item_1)

In [ ]:
duemila_disney_file

In [50]:
# codice alejandro correlazione
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from tsr_functions import *
from sklearn.preprocessing import MultiLabelBinarizer

In [51]:
def get_relevant_words(X, y, nwords, featnames, tsr_function=information_gain_positive):
    nD,nF = X.shape
    positives = y.sum()
    negatives = nD - positives

    # computes the 4-cell contingency tables for each feature
    TP = np.asarray((X[y==1]>0).sum(axis=0)).flatten()
    FN = positives - TP
    FP = np.asarray((X[y==0]>0).sum(axis=0)).flatten()
    TN = negatives - FP
    _4cell = [ContTable(tp=TP[i], tn=TN[i], fp=FP[i], fn=FN[i]) for i in range(nF)]

    # applies the tsr_function to the 4-cell counters
    feat_informativeness = np.array(list(map(tsr_function, _4cell)))

    top_relevant_terms = np.argsort(-feat_informativeness)[:nwords]
    feat_names = np.asarray(featnames)[top_relevant_terms]
    feat_informativeness = feat_informativeness[top_relevant_terms]

    return list(zip(feat_names, feat_informativeness))


In [52]:
documents = data
target = np.array(t)

In [53]:
len(target)

432

In [54]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, stop_words = STOPWORDS)
X = tfidf.fit_transform(data)

C:\Users\Attilio\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aaaaaaahhhhh'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [55]:
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(target.reshape(-1, 1))
nC = Y.shape[1]
print(Y.shape)

(432, 41)


In [56]:
nwords = 100
for i in range(nC):
    # getting the 10 most relevant terms for each class according to the (positive only) correlation to the
    # group label, as quantified by information gain
    class_i_relevant = get_relevant_words(X, Y[:,i], nwords, tfidf.get_feature_names())
    one_doc_as_df = pd.DataFrame.from_records(class_i_relevant)
    one_doc_as_df.to_csv(f'class {i}.txt', header = None, index = False)
    #print(f'class {i}: ' + ', '.join([f'{word} ({rel:.3f})' for word, rel in class_i_relevant]))
    #print('#############################################')
    print(f'class {i} done')

class 0 done
class 1 done
class 2 done
class 3 done
class 4 done
class 5 done
class 6 done
class 7 done
class 8 done
class 9 done
class 10 done
class 11 done
class 12 done
class 13 done
class 14 done
class 15 done
class 16 done
class 17 done
class 18 done
class 19 done
class 20 done
class 21 done
class 22 done
class 23 done
class 24 done
class 25 done
class 26 done
class 27 done
class 28 done
class 29 done
class 30 done
class 31 done
class 32 done
class 33 done
class 34 done
class 35 done
class 36 done
class 37 done
class 38 done
class 39 done
class 40 done


In [ ]:
one_doc_as_df.to_csv('prova.txt', header=None, index=False)

In [ ]:
pd.DataFrame.from_records(class_i_relevant,  columns=['term', 'score'])